In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
import numpy as np
import tensorflow as tf
path = r'C:\\Users\\weso\\Desktop\\glove.6B\\'
id_dict = {}
word_dict = {}

matrix = []
dimensions = 300
with open(path + "glove.6B.300d.txt", 'r' , encoding="utf-8")  as f:
    lines = f.readlines()
    vocab_size = len(lines)
    
    matrix = np.zeros((vocab_size,dimensions),dtype=float)
    for line in lines:
        values = line.split()
        word = values[0].strip()
        id = len(id_dict)
        id_dict[word]=id
        word_dict[id] = word
        vector = np.asarray(values[1:], "double")
        matrix[id_dict[word],:] = vector
print(len(id_dict))

400000


In [3]:
print(type(word_dict))

<class 'dict'>


## 3cosAdd 

In [4]:
matrix_normalized = tf.nn.l2_normalize(matrix,axis = 1)
def three_cos_predict_np(a,a_star,b,b_star):
    if( a in id_dict and b in id_dict and a_star in id_dict and b_star in id_dict):
        pass
    else:
        return None
    id_a = id_dict[a]
    id_b = id_dict[b]
    id_a_star = id_dict[a_star]
    id_b_star = id_dict[b_star]#remove this after testing
    weight_a = matrix[id_a,:]
    weight_b = matrix[id_b,:]
    weight_a_star = matrix[id_a_star,:]
    direction = weight_b + ( weight_a_star - weight_a)
    direction /= np.linalg.norm(direction)
    
    sim = tf.tensordot(tf.convert_to_tensor(matrix_normalized),tf.convert_to_tensor(direction),axes = 1)
    myList = [id_a, id_b, id_a_star];
    myList.sort();
    index1 = np.argmax(sim[:myList[0]])
    sim2 = sim[myList[0]+1:myList[1]]
    if len(sim2) > 0:
        index2 = np.argmax(sim[myList[0]+1:myList[1]])
        index2 +=  myList[0]+1
    else:
        index2 = None
    sim3 = sim[myList[1]+1:myList[2]]
    if len(sim3) > 0:
        index3 = np.argmax(sim[myList[1]+1:myList[2]])
        index3 +=  myList[1]+1
    else:
        index3 = None
    index4 = np.argmax(sim[myList[2]+1:])
    index4 +=  myList[2]+1
    
    myList = [index1, index2, index3,index4];
    values = []
    myList2 = []
    for index in myList:
        if index != None:
            values.append(sim[index])
            myList2.append(index)
    index = myList2[np.argmax(values)]
    
    return word_dict[index]

In [5]:
import time
startTime = time.time()

print(three_cos_predict_np('small','smaller','weak','weaker'))
print(three_cos_predict_np('big','bigger','small','smaller'))
print(three_cos_predict_np('germany','berlin','france','paris'))
print(three_cos_predict_np('one','two','three','four'))

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

weaker
larger
paris
four
Execution time in seconds: 0.22805190086364746


In [7]:
count_sucess = 0
count_unsucessful = 0
count_out_of_vocab = 0
count_b = 0
with open(r'..\datasets\miklov Analogies.txt') as file:
    lines = file.readlines()
    for index, line in enumerate(lines):
        if(index % 1000 == 0):
            print(str(index) + " of " + str(len(lines)))
            
        if line.startswith(":"):
            pass
        else:
            a,a_star,b,b_star = line.split(" ")
            b_star = b_star.strip()#super important
            
            predicted = three_cos_predict_np(a,a_star,b,b_star)
            if(predicted == None):
                count_out_of_vocab += 1
            elif(predicted == b_star):
                count_sucess += 1
            elif(predicted == b):
                count_b += 1
            else:
                count_unsucessful += 1

print(count_sucess/ len(lines))

0 of 13106
1000 of 13106
2000 of 13106
3000 of 13106
4000 of 13106
5000 of 13106
6000 of 13106
7000 of 13106
8000 of 13106
9000 of 13106
10000 of 13106
11000 of 13106
12000 of 13106
13000 of 13106
0.724858843277888


In [8]:
import winsound
winsound.Beep(440, 500)

normalised before exclude

In [10]:
def three_cos_predict_np_norm(a,a_star,b,b_star):
    if( a in id_dict and b in id_dict and a_star in id_dict and b_star in id_dict):
        pass
    else:
        return None
    id_a = id_dict[a]
    id_b = id_dict[b]
    id_a_star = id_dict[a_star]
    id_b_star = id_dict[b_star]#remove this after testing
    weight_a = matrix_normalized[id_a,:]
    weight_b = matrix_normalized[id_b,:]
    weight_a_star = matrix_normalized[id_a_star,:]
    direction = weight_b + ( weight_a_star - weight_a)
    direction /= np.linalg.norm(direction)
    
    sim = tf.tensordot(tf.convert_to_tensor(matrix_normalized),tf.convert_to_tensor(direction),axes = 1)
    myList = [id_a, id_b, id_a_star];
    myList.sort();
    index1 = np.argmax(sim[:myList[0]])
    sim2 = sim[myList[0]+1:myList[1]]
    if len(sim2) > 0:
        index2 = np.argmax(sim[myList[0]+1:myList[1]])
        index2 +=  myList[0]+1
    else:
        index2 = None
    sim3 = sim[myList[1]+1:myList[2]]
    if len(sim3) > 0:
        index3 = np.argmax(sim[myList[1]+1:myList[2]])
        index3 +=  myList[1]+1
    else:
        index3 = None
    index4 = np.argmax(sim[myList[2]+1:])
    index4 +=  myList[2]+1
    
    myList = [index1, index2, index3,index4];
    values = []
    myList2 = []
    for index in myList:
        if index != None:
            values.append(sim[index])
            myList2.append(index)
    index = myList2[np.argmax(values)]
    
    return word_dict[index]

In [12]:
count_sucess = 0
count_unsucessful = 0
count_out_of_vocab = 0
count_b = 0
with open(r'..\datasets\miklov Analogies.txt') as file:
    lines = file.readlines()
    for index, line in enumerate(lines):
        if(index % 1000 == 0):
            print(str(index) + " of " + str(len(lines)))
            
        if line.startswith(":"):
            pass
        else:
            a,a_star,b,b_star = line.split(" ")
            b_star = b_star.strip()#super important
            
            predicted = three_cos_predict_np_norm(a,a_star,b,b_star)
            if(predicted == None):
                count_out_of_vocab += 1
            elif(predicted == b_star):
                count_sucess += 1
            elif(predicted == b):
                count_b += 1
            else:
                count_unsucessful += 1

print(count_sucess/ len(lines))

0 of 13106
1000 of 13106
2000 of 13106
3000 of 13106
4000 of 13106
5000 of 13106
6000 of 13106
7000 of 13106
8000 of 13106
9000 of 13106
10000 of 13106
11000 of 13106
12000 of 13106
13000 of 13106
0.7239432321074317


## 3cos Mult

In [13]:
def sim_01(matrix,weight):
    tmp = tf.tensordot(matrix,weight,axes=1)
    tmp = (tmp+1)/2.0
    return tmp

def predict_three_cos_mult(a,a_star,b,b_star):
    if( a in id_dict and b in id_dict and a_star in id_dict and b_star in id_dict):
        pass
    else:
        return None
    
    id_a = id_dict[a]
    id_b = id_dict[b]
    id_a_star = id_dict[a_star]
    
    weight_a = matrix_normalized[id_a,:]
    weight_b = matrix_normalized[id_b,:]
    weight_a_star = matrix_normalized[id_a_star,:]
    
    nominator = sim_01(matrix_normalized,weight_a_star)
    
    nominator = nominator * sim_01(matrix_normalized,weight_b) 
    donominator = sim_01(matrix_normalized,weight_a) + 0.0001
    sim = nominator / donominator
    myList = [id_a, id_b, id_a_star];
    myList.sort();
    index1 = np.argmax(sim[:myList[0]])
    sim2 = sim[myList[0]+1:myList[1]]
    if len(sim2) > 0:
        index2 = np.argmax(sim[myList[0]+1:myList[1]])
        index2 +=  myList[0]+1
    else:
        index2 = None
    sim3 = sim[myList[1]+1:myList[2]]
    if len(sim3) > 0:
        index3 = np.argmax(sim[myList[1]+1:myList[2]])
        index3 +=  myList[1]+1
    else:
        index3 = None
    index4 = np.argmax(sim[myList[2]+1:])
    index4 +=  myList[2]+1
    
    myList = [index1, index2, index3,index4];
    values = []
    myList2 = []
    for index in myList:
        if index != None:
            values.append(sim[index])
            myList2.append(index)
    index = myList2[np.argmax(values)]
    
    return word_dict[index]

In [14]:
import time
startTime = time.time()

print(predict_three_cos_mult('small','smaller','weak','weaker'))
print(predict_three_cos_mult('big','bigger','small','smaller'))
print(predict_three_cos_mult('germany','berlin','france','paris'))
print(predict_three_cos_mult('one','two','three','four'))

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

weaker
larger
paris
four
Execution time in seconds: 0.6551494598388672


In [15]:
count_sucess = 0
count_unsucessful = 0
count_out_of_vocab = 0
with open(r'..\datasets\miklov Analogies.txt') as file:
    lines = file.readlines()
    for index, line in enumerate(lines):
        if(index % 100 == 0):
            print(str(index) + " of " + str(len(lines)) +" => "+str( count_sucess))
            
        if line.startswith(":"):
            pass
        else:
            a,a_star,b,b_star = line.split(" ")
            b_star = b_star.strip()#super important
            
            predicted = predict_three_cos_mult(a,a_star,b,b_star)
            if(predicted == None):
                count_out_of_vocab += 1
            elif(predicted == b_star):
                count_sucess += 1
            else:
                count_unsucessful += 1
print(count_sucess,count_unsucessful)
print(count_sucess/count_unsucessful * 100)

0 of 13106 => 0
100 of 13106 => 98
200 of 13106 => 193
300 of 13106 => 287
400 of 13106 => 385
500 of 13106 => 477
600 of 13106 => 575
700 of 13106 => 669
800 of 13106 => 762
900 of 13106 => 854
1000 of 13106 => 951
1100 of 13106 => 1047
1200 of 13106 => 1143
1300 of 13106 => 1239
1400 of 13106 => 1337
1500 of 13106 => 1434
1600 of 13106 => 1526
1700 of 13106 => 1619
1800 of 13106 => 1716
1900 of 13106 => 1812
2000 of 13106 => 1909
2100 of 13106 => 2007
2200 of 13106 => 2101
2300 of 13106 => 2197
2400 of 13106 => 2287
2500 of 13106 => 2381
2600 of 13106 => 2478
2700 of 13106 => 2577
2800 of 13106 => 2674
2900 of 13106 => 2773
3000 of 13106 => 2866
3100 of 13106 => 2963
3200 of 13106 => 3057
3300 of 13106 => 3151
3400 of 13106 => 3249
3500 of 13106 => 3345
3600 of 13106 => 3438
3700 of 13106 => 3535
3800 of 13106 => 3629
3900 of 13106 => 3724
4000 of 13106 => 3821
4100 of 13106 => 3917
4200 of 13106 => 4012
4300 of 13106 => 4108
4400 of 13106 => 4205
4500 of 13106 => 4303
4600 of 13106 

In [17]:
9442 / 13106 

0.7204333892873493

In [16]:
import winsound
winsound.Beep(440, 500)